In [2]:
import glob
import os
import random
import sys

In [3]:
import numpy as np

In [4]:
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from utils.npz_files import open_npz, save_npz
from utils.features import compute_cwru_features
from utils.transform import extract_sequences

In [5]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [6]:
RPMS_LIST = [1730, 1750, 1772, 1797]
FAULT_LIST = ['IR', 'B', 'OR@6', 'OR@3', 'OR@12']
DIAMETER_LIST = [7, 14, 21, 28]
END_LIST = ['FE', 'DE12', 'DE48']

## Open normal data

In [7]:
from tqdm import tqdm

In [8]:
ROOT_DATA_DIR = '../Data'
ROOT_DATA_SERIES_DIR = '../datasets/step1'

for item in os.walk(ROOT_DATA_DIR):
    input_directory = item[0]
    output_directory = input_directory.replace(ROOT_DATA_DIR, ROOT_DATA_SERIES_DIR)
    os.makedirs(output_directory, exist_ok=True)
    
    for basefile in tqdm(item[2]):
        if os.path.splitext(basefile)[1] != '.npz':
            continue

        input_filepath = os.path.join(input_directory, basefile)
        output_filepath = os.path.join(output_directory, basefile)
        
        # print((input_filepath, output_filepath))
        data = open_npz(input_filepath)
        output_data = {}
        for k in data.keys():
            if k == 'BA': continue # ignore BA
            len_time_series = data[k].shape[0]
            number_rows = 8192 if "normal" in input_filepath.lower() else 1024
            transformed_data = extract_sequences(data[k], number_rows, 2048, random_seed=RANDOM_SEED)
            output_data[k] = transformed_data
        
        save_npz(output_filepath, **output_data)

0it [00:00, ?it/s]
100%|██████████| 40/40 [00:03<00:00, 11.23it/s]
